In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score, log_loss
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import dask.dataframe as dd
import optuna
import sys
#sys.path.append('./')

#from optimizer import tune_XGBoost

In [2]:
df = dd.read_parquet('G:\RecSysChallenge2021\Sample_dataset')

In [3]:
def count_token(ds,token):
    not_null = (ds.isnull()==0).astype('int32')
    #print('type of not null variable:', not_null.dtypes)
    return (((ds.str.count(token)+1)*not_null).fillna(0)).astype('int64')

In [4]:
texts = df['raw_feature_tweet_text_token'].head()
count_token(texts,'\t')

0     60
1     30
2     67
3    104
4    104
Name: raw_feature_tweet_text_token, dtype: int64

In [5]:
%%time

df = df.drop(columns=['mapped_creator_id', 'mapped_engager_id', 'mapped_tweet_id'])

df['number_of_tokens'] = df['raw_feature_tweet_text_token'].map_partitions(lambda ds: count_token(ds,'\t'), meta='int')
df = df.drop(columns=['raw_feature_tweet_text_token'])

df['number_of_hashtags'] = df['mapped_tweet_hashtags_id'].map(lambda x: len(x))
df['number_of_domains'] = df['mapped_domains_id'].map(lambda x: len(x))
df['number_of_links'] = df['mapped_tweet_links_id'].map(lambda x: len(x))
df = df.drop(columns=['mapped_tweet_hashtags_id', 'mapped_domains_id', 'mapped_tweet_links_id'])

df['date'] = df['tweet_timestamp'].astype('datetime64[s]')
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df = df.drop(columns=['tweet_timestamp', 'date'])

df['creator_creation_date'] = df['creator_creation_timestamp'].astype('datetime64[s]')
df['creator_creation_month'] = df['creator_creation_date'].dt.month
df['creator_creation_year'] = df['creator_creation_date'].dt.year 
df = df.drop(columns=['creator_creation_timestamp','creator_creation_date'])

df['engager_creation_date'] = df['engager_creation_timestamp'].astype('datetime64[s]')
df['engager_creation_month'] = df['engager_creation_date'].dt.month
df['engager_creation_year'] = df['engager_creation_date'].dt.year 
df = df.drop(columns=['engager_creation_timestamp', 'engager_creation_date'])

df['creator_ratio'] = df['creator_follower_count'] / (df['creator_following_count']+1)
df['engager_ratio'] = df['engager_follower_count'] / (df['engager_following_count']+1)

df['engagement_reply_timestamp'] = df['engagement_reply_timestamp'].map({-1: 0}).fillna(1)
df['engagement_retweet_timestamp'] = df['engagement_retweet_timestamp'].map({-1: 0}).fillna(1)
df['engagement_comment_timestamp'] = df['engagement_comment_timestamp'].map({-1: 0}).fillna(1)
df['engagement_like_timestamp'] = df['engagement_like_timestamp'].map({-1: 0}).fillna(1)

# Keep only likes
df = df.drop(columns=['engagement_reply_timestamp', 'engagement_retweet_timestamp', 'engagement_comment_timestamp'])

Wall time: 188 ms


In [6]:
parted_df = df.repartition(npartitions=8)

In [7]:
part = parted_df
part = part.compute()

In [8]:
part.dtypes

creator_follower_count                  int32
creator_following_count                 int32
creator_is_verified                      bool
engager_follower_count                  int32
engager_following_count                 int32
engager_is_verified                      bool
engagement_creator_follows_engager       bool
engagement_like_timestamp             float64
number_of_photo                         uint8
number_of_gif                           uint8
number_of_video                         uint8
mapped_language_id                      uint8
mapped_tweet_type                       uint8
number_of_tokens                        int64
number_of_hashtags                      int64
number_of_domains                       int64
number_of_links                         int64
hour                                    int64
day                                     int64
day_of_week                             int64
creator_creation_month                  int64
creator_creation_year             

In [11]:
part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10145968 entries, 0 to 259216
Data columns (total 25 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   creator_follower_count              int32  
 1   creator_following_count             int32  
 2   creator_is_verified                 bool   
 3   engager_follower_count              int32  
 4   engager_following_count             int32  
 5   engager_is_verified                 bool   
 6   engagement_creator_follows_engager  bool   
 7   number_of_photo                     uint8  
 8   number_of_gif                       uint8  
 9   number_of_video                     uint8  
 10  mapped_language_id                  uint8  
 11  mapped_tweet_type                   uint8  
 12  number_of_tokens                    int64  
 13  number_of_hashtags                  int64  
 14  number_of_domains                   int64  
 15  number_of_links                     int64  
 16  

In [10]:
part['engagement_like_timestamp'] = part['engagement_like_timestamp'].astype(np.int8)
y = part['engagement_like_timestamp']
part.drop('engagement_like_timestamp', axis=1, inplace=True)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(part, y, stratify=y, test_size=0.15)

In [14]:
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, stratify=y_val, test_size=0.5)

In [15]:
X_train.creator_follower_count

135341    179501
180768       201
200541      1944
256122     66359
12158     279075
           ...  
17348       1874
142701    317531
7325       36120
111761    174600
101382    177758
Name: creator_follower_count, Length: 8624072, dtype: int32

In [16]:
import gc
gc.collect()

968

In [13]:
bst = tune_XGBoost(X_train, X_val, y_train, y_val, 2)

[I 2021-05-15 18:29:43,904] A new study created in memory with name: no-name-99204112-9525-4b4f-86f2-4efbcd5dfc13
[I 2021-05-15 18:31:39,294] Trial 0 finished with values: [1.7152674960026038, 0.5511167736170909] and parameters: {'booster': 'dart', 'lambda': 0.019763208036631633, 'alpha': 1.7398245729828649e-06, 'subsample': 0.893445716579504, 'colsample_bytree': 0.7517110853975175, 'max_depth': 5, 'min_child_weight': 4, 'eta': 0.0369735414634237, 'gamma': 0.002575861449977913, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 1.5077185272991423e-08, 'skip_drop': 1.10881639851498e-06}. 
[I 2021-05-15 18:33:42,043] Trial 1 finished with values: [-3.2806739565085508, 0.5757587151454591] and parameters: {'booster': 'dart', 'lambda': 1.280921625937904e-07, 'alpha': 2.6811798576234075e-05, 'subsample': 0.2305590798116123, 'colsample_bytree': 0.5188417535730951, 'max_depth': 9, 'min_child_weight': 5, 'eta': 8.163576074464615e-06, 'gamma': 0.1084213

In [27]:
#LightGBM optimizer

import optuna
from sklearn.metrics import average_precision_score
import xgboost as xgb
import optuna.integration.lightgbm as lgb_sequential
import lightgbm as lgb
import catboost as cb
 

def tune_lightGBM_naive(X_train, X_val, y_train, y_val, n_trials):
    '''
    Simple tuning of lightGBM using the standard Optuna approach. It basically defines a space of parameters and optuna
    search for the optimal value in that space doing combination of parameters values. 
    
    https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258
    More info here about naive vs sequential lightGBM
    
    This directly return the model that maximizes the given metrics (rce and avg_precision)
    '''

    class Objective_lightGBM_naive(object):
        def __init__(self, X_train, X_val, y_train, y_val):
            # Hold this implementation specific arguments as the fields of the class.
            self.X_train, self.X_val, self.y_train, self.y_val = X_train, X_val, y_train, y_val
            self.dtrain = lgb.Dataset(self.X_train, label=self.y_train)


        def calculate_ctr(self, gt):
            positive = len([x for x in gt if x == 1])
            ctr = positive/float(len(gt))
            return ctr

        def compute_rce(self, pred, gt):
            cross_entropy = log_loss(gt, pred)
            data_ctr = self.calculate_ctr(gt)
            strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
            return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

        def __call__(self, trial):
            # Calculate an objective value by using the extra arguments.

            self.param = {
                'objective': 'binary',
                'metric': 'binary_logloss',
                'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
                'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
                'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'feature_pre_filter': False,
                'verbose':2
            }


            gbm = lgb.train(self.param, self.dtrain)
            preds = gbm.predict(self.X_val)
            #pred_labels = np.rint(preds) #qui arrotonda all'intero più vicino 
            rce = self.compute_rce(preds, self.y_val)
            avg_precision = average_precision_score(self.y_val, preds)
            return rce, avg_precision

    # Execute an optimization by using an `Objective` instance.
    study = optuna.create_study(directions=["maximize", "maximize"])
    study.optimize(Objective_lightGBM_naive(X_train, X_val, y_train, y_val), n_trials=n_trials)
    return study.best_trials

    
def tune_lightGBM_sequential(X_train, X_val, y_train, y_val):
    '''
    This function tunes the lightGBM method in an incremental way on the following hyper-parameters:
    ``lambda_l1``, ``lambda_l2``, ``num_leaves``, ``feature_fraction``, ``bagging_fraction``,
    ``bagging_freq`` and ``min_child_samples``.
    
    The code will show both the output of the lightGBM training and the optuna output.
    
    The "single" steps depending on the num_boost_round or early_stopping_rounds come from the lightGBM actual training.
    Once these steps are completed, the model is evaluated and a new trial begins. Trials come from the optuna library.
    The number of trials for each hyper-parameter tuning has been set empirically inside the library, you can find more here:
    https://optuna.readthedocs.io/en/stable/_modules/optuna/integration/_lightgbm_tuner/optimize.html#LightGBMTuner by searching 
    for "n_trials"
    
    NB: I have been able to set custom metrics for the lightGBM train, however it seems that optuna takes into consideration only
    the log-loss to select the best model between the different trials. This behaviour needs further investigation 
    
    '''
    
    def calculate_ctr(gt):
        positive = len([x for x in gt if x == 1])
        ctr = positive/float(len(gt))
        return ctr

    def compute_rce(preds, train_data):
        gt = train_data.get_label()
        cross_entropy = log_loss(gt, preds)
        data_ctr = calculate_ctr(gt)
        strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
        rce = (1.0 - cross_entropy/strawman_cross_entropy)*100.0
        return ('rce', rce, True)

    def compute_avg_precision(preds, train_data):
        gt = train_data.get_label()
        avg_precision= average_precision_score(gt, preds)
        return('avg_precision', avg_precision, True)
    
    params = {
        "objective": "binary",
        "metric": 'binary_logloss',
        "boosting_type": "gbdt",
        "verbose" : 2
    }
    
    dtrain = lgb_sequential.Dataset(X_train, label=y_train)
    dval = lgb_sequential.Dataset(X_val, label=y_val)
    
    print('Starting training lightGBM sequential')
    model = lgb_sequential.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=True,num_boost_round =2, early_stopping_rounds=100, feval = [compute_rce, compute_avg_precision]
    )
    
    return model.params


def tune_XGBoost(X_train, X_val, y_train, y_val, n_trials):
    '''
    Simple tuning of lightGBM using the standard Optuna approach. It basically defines a space of parameters and optuna
    search for the optimal value in that space doing combination of parameters values. 
    
    https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258
    More info here about naive vs sequential lightGBM
    
    This directly return the model that maximizes the given metrics (rce and avg_precision)
    '''

    #Per XGBoost la metrica di training più sensata sembra essere la binary:logloss. Poi non devo far alcuna validation strana,
    #semplicemente predico, faccio il comparison con le label e calcolo la RCE e AVG_Precision. Molto straight forward. 
    class Objective_XGBoost(object):
        def __init__(self, X_train, X_val, y_train, y_val):
            # Hold this implementation specific arguments as the fields of the class.
            self.X_train, self.X_val, self.y_train, self.y_val = X_train, X_val, y_train, y_val
            self.dtrain = xgb.DMatrix(self.X_train, label=self.y_train)
            self.dvalid = xgb.DMatrix(self.X_val, label=self.y_val)


        def calculate_ctr(self, gt):
            positive = len([x for x in gt if x == 1])
            ctr = positive/float(len(gt))
            return ctr

        def compute_rce(self, pred, gt):
            cross_entropy = log_loss(gt, pred)
            data_ctr = self.calculate_ctr(gt)
            strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
            return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

        def __call__(self, trial):
            # Calculate an objective value by using the extra arguments.

            self.param = {
                "verbosity": 0,
                "objective": "binary:logistic",
                # use exact for small dataset.
                "tree_method": "exact",
                # defines booster, gblinear for linear functions.
                "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
                # L2 regularization weight.
                "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                # L1 regularization weight.
                "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
                # sampling ratio for training data.
                "subsample": trial.suggest_float("subsample", 0.2, 1.0),
                # sampling according to each tree.
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
            }

            if self.param["booster"] in ["gbtree", "dart"]:
                # maximum depth of the tree, signifies complexity of the tree.
                self.param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
                # minimum child weight, larger the term more conservative the tree.
                self.param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
                self.param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
                # defines how selective algorithm is.
                self.param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
                self.param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

            if self.param["booster"] == "dart":
                self.param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
                self.param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
                self.param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
                self.param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)


            bst = xgb.train(self.param, self.dtrain)
            preds = bst.predict(self.dvalid)
            #pred_labels = np.rint(preds) #qui arrotonda all'intero più vicino 
            rce = self.compute_rce(preds, self.y_val)
            avg_precision = average_precision_score(self.y_val, preds)
            return rce, avg_precision

    # Execute an optimization by using an `Objective` instance.
    study = optuna.create_study(directions=["maximize", "maximize"])
    study.optimize(Objective_XGBoost(X_train, X_val, y_train, y_val), n_trials=n_trials)
    return study.best_trials

def tune_CatBoost(X_train, X_val, y_train, y_val, n_trials):
    '''
    Simple tuning of lightGBM using the standard Optuna approach. It basically defines a space of parameters and optuna
    search for the optimal value in that space doing combination of parameters values. 
    
    https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258
    More info here about naive vs sequential lightGBM
    
    This directly return the model that maximizes the given metrics (rce and avg_precision)
    '''

    class Objective_CatBoost(object):
        def __init__(self, X_train, X_val, y_train, y_val):
            # Hold this implementation specific arguments as the fields of the class.
            self.X_train, self.X_val, self.y_train, self.y_val = X_train, X_val, y_train, y_val


        def calculate_ctr(self, gt):
            positive = len([x for x in gt if x == 1])
            ctr = positive/float(len(gt))
            return ctr

        def compute_rce(self, pred, gt):
            cross_entropy = log_loss(gt, pred)
            data_ctr = self.calculate_ctr(gt)
            strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
            return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

        def __call__(self, trial):
            # Calculate an objective value by using the extra arguments.

            self.param = {
                    "objective": trial.suggest_categorical("objective", ["Logloss"]),
                    #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
                    "depth": trial.suggest_int("depth", 1, 12),
                    "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
                    "bootstrap_type": trial.suggest_categorical(
                        "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
                    ),
                    "used_ram_limit": "8gb",
                    "task_type": "GPU"
                }

            if self.param["bootstrap_type"] == "Bayesian":
                self.param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
            elif self.param["bootstrap_type"] == "Bernoulli":
                self.param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

            gbm = cb.CatBoostClassifier(**self.param)
            gbm.fit(self.X_train, self.y_train, eval_set=[(self.X_val, self.y_val)], verbose=2, early_stopping_rounds=2)
            print('classes:',gbm.classes_)
            preds = gbm.predict_proba(data=self.X_val)
            print('printing shape of prediction:',preds.shape)
            #pred_labels = np.rint(preds) #qui arrotonda all'intero più vicino 
            rce = self.compute_rce(preds[:,1], self.y_val)
            avg_precision = average_precision_score(self.y_val, preds[:,1])
            return rce, avg_precision

    # Execute an optimization by using an `Objective` instance.
    study = optuna.create_study(directions=["maximize", "maximize"])
    study.optimize(Objective_CatBoost(X_train, X_val, y_train, y_val), n_trials=n_trials)
    return study



In [54]:
lightGBM_naive = tune_lightGBM_naive(X_train, X_val, y_train, y_val, 3)

[I 2021-05-19 12:00:21,157] A new study created in memory with name: no-name-c57ce5d1-7376-4356-9a04-034f0f565fdd


[LightGBM] [Info] Number of positive: 2790775, number of negative: 4266576
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.870801
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.310365
[LightGBM] [Debug] init for col-wise cost 0.323099 seconds, init for row-wise cost 1.315928 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.534536 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1982
[LightGBM] [Info] Number of data points in the train set: 7057351, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395442 -> initscore=-0.424492
[LightGBM] [Info] Start training from score -0.424492
[LightGBM] [Debug] Re-bagging, using 5243804 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 8
[Light

[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 7
[LightGBM] [Debug] Re-bagging, using 5240925 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 8
[LightGBM] [Debug] Re-bagging, using 5239868 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 8
[LightGBM] [Debug] Re-bagging, using 5240263 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 9
[LightGBM] [Debug] Re-bagging, using 5240681 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 12
[LightGBM] [Debug] Re-bagging, using 5240090 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 9
[LightGBM] [Debug] Re-bagging, using 5241278 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 8
[LightGBM] [Debug] Re-bagging, using 5240047 data to train
[LightGBM] [Debug] Trained a tree with leaves = 16 and depth = 8
[LightGBM] [Debug] Re-bagging, using 5235769 data to train
[LightG

[I 2021-05-19 12:01:33,098] Trial 0 finished with values: [9.680037022066978, 0.590107126869458] and parameters: {'lambda_l1': 0.00016003343394497234, 'lambda_l2': 7.253474962847375e-08, 'num_leaves': 16, 'feature_fraction': 0.9203362753514731, 'bagging_fraction': 0.7425350716049339, 'bagging_freq': 1, 'min_child_samples': 50}. 


[LightGBM] [Info] Number of positive: 2790775, number of negative: 4266576
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.870801
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.310365
[LightGBM] [Debug] init for col-wise cost 0.401325 seconds, init for row-wise cost 1.533451 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.688320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1982
[LightGBM] [Info] Number of data points in the train set: 7057351, number of used features: 25
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395442 -> initscore=-0.424492
[LightGBM] [Info] Start training from score -0.424492
[LightGBM] [Debug] Re-bagging, using 3092087 data to train
[LightGBM] [Debug] Trained a 

[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 16
[LightGBM] [Debug] Re-bagging, using 3090498 data to train
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 23
[LightGBM] [Debug] Re-bagging, using 3085339 data to train
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 18
[LightGBM] [Debug] Re-bagging, using 3090110 data to train
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 202 and depth = 22
[LightGBM] [Debug] 

[I 2021-05-19 12:03:17,009] Trial 1 finished with values: [11.445588023120889, 0.6130723365173631] and parameters: {'lambda_l1': 0.003742329399403925, 'lambda_l2': 0.38040375783046276, 'num_leaves': 202, 'feature_fraction': 0.9148439542481932, 'bagging_fraction': 0.43766517235927777, 'bagging_freq': 3, 'min_child_samples': 76}. 


[LightGBM] [Info] Number of positive: 2790775, number of negative: 4266576
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.870801
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.310365
[LightGBM] [Debug] init for col-wise cost 0.406622 seconds, init for row-wise cost 1.560347 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.704871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1982
[LightGBM] [Info] Number of data points in the train set: 7057351, number of used features: 25
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395442 -> initscore=-0.424492
[LightGBM] [Info] Start training from score -0.424492
[LightGBM] [Debug] Re-bagging, using 3174979 data to train
[LightGBM] [Debug] Trained a 

[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 18
[LightGBM] [Debug] Re-bagging, using 3171148 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 11
[LightGBM] [Debug] Re-bagging, using 3171876 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 17
[LightGBM] [Debug] Re-bagging, using 3171555 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 16
[LightGBM] [Debug] Re-bagging, using 3171651 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 15
[LightGBM] [Debug] Re-bagging, using 3171079 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 20
[LightGBM] [Debug] Re-bagging, using 3173048 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 14
[LightGBM] [Debug] Re-bagging, using 3167969 data to train
[LightGBM] [Debug] Trained a tree with leaves = 74 and depth = 15
[LightGBM] [Debug] Re-bagging, using 3170906 data to train


[I 2021-05-19 12:05:15,009] Trial 2 finished with values: [10.798041756039456, 0.6049990022746099] and parameters: {'lambda_l1': 1.7751325963368778, 'lambda_l2': 0.02677069115641172, 'num_leaves': 74, 'feature_fraction': 0.8905491418172672, 'bagging_fraction': 0.4493832308113309, 'bagging_freq': 1, 'min_child_samples': 54}. 


In [29]:
%%time

study = tune_CatBoost(X_train, X_val, y_train, y_val, n_trials=30)

optuna.visualization.plot_pareto_front(study, target_names=["RCE", "AP"])

[I 2021-05-20 11:18:53,981] A new study created in memory with name: no-name-1bae1804-634d-40f6-a808-d1f15e4f021d


Learning rate set to 0.03296
0:	learn: 0.6890707	test: 0.6890768	best: 0.6890768 (0)	total: 49.9ms	remaining: 49.8s
2:	learn: 0.6817361	test: 0.6817503	best: 0.6817503 (2)	total: 146ms	remaining: 48.5s
4:	learn: 0.6753709	test: 0.6753921	best: 0.6753921 (4)	total: 237ms	remaining: 47.2s
6:	learn: 0.6698458	test: 0.6698756	best: 0.6698756 (6)	total: 332ms	remaining: 47.1s
8:	learn: 0.6650518	test: 0.6650891	best: 0.6650891 (8)	total: 424ms	remaining: 46.7s
10:	learn: 0.6608944	test: 0.6609368	best: 0.6609368 (10)	total: 516ms	remaining: 46.4s
12:	learn: 0.6572897	test: 0.6573395	best: 0.6573395 (12)	total: 609ms	remaining: 46.2s
14:	learn: 0.6541591	test: 0.6542131	best: 0.6542131 (14)	total: 701ms	remaining: 46s
16:	learn: 0.6514421	test: 0.6514999	best: 0.6514999 (16)	total: 792ms	remaining: 45.8s
18:	learn: 0.6490819	test: 0.6491445	best: 0.6491445 (18)	total: 884ms	remaining: 45.6s
20:	learn: 0.6470351	test: 0.6471014	best: 0.6471014 (20)	total: 974ms	remaining: 45.4s
22:	learn: 0.6

[I 2021-05-20 11:19:14,984] Trial 0 finished with values: [6.21461492382529, 0.5341421945140188] and parameters: {'objective': 'Logloss', 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. 


Learning rate set to 0.03296
0:	learn: 0.6893117	test: 0.6893148	best: 0.6893148 (0)	total: 44.3ms	remaining: 44.3s
2:	learn: 0.6824099	test: 0.6824188	best: 0.6824188 (2)	total: 128ms	remaining: 42.6s
4:	learn: 0.6764193	test: 0.6764367	best: 0.6764367 (4)	total: 210ms	remaining: 41.9s
6:	learn: 0.6712265	test: 0.6712480	best: 0.6712480 (6)	total: 297ms	remaining: 42.1s
8:	learn: 0.6667195	test: 0.6667479	best: 0.6667479 (8)	total: 381ms	remaining: 42s
10:	learn: 0.6628168	test: 0.6628484	best: 0.6628484 (10)	total: 461ms	remaining: 41.5s
12:	learn: 0.6594336	test: 0.6594712	best: 0.6594712 (12)	total: 541ms	remaining: 41s
14:	learn: 0.6565018	test: 0.6565406	best: 0.6565406 (14)	total: 619ms	remaining: 40.7s
16:	learn: 0.6539525	test: 0.6539975	best: 0.6539975 (16)	total: 698ms	remaining: 40.4s
18:	learn: 0.6517426	test: 0.6517892	best: 0.6517892 (18)	total: 777ms	remaining: 40.1s
20:	learn: 0.6498190	test: 0.6498679	best: 0.6498679 (20)	total: 854ms	remaining: 39.8s
22:	learn: 0.648

[I 2021-05-20 11:19:35,006] Trial 1 finished with values: [5.0798619023436515, 0.47942419232746336] and parameters: {'objective': 'Logloss', 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. 


...

724:	learn: 0.6052706	test: 0.6056054	best: 0.6056054 (724)	total: 3m 32s	remaining: 1m 20s
726:	learn: 0.6052601	test: 0.6055957	best: 0.6055957 (726)	total: 3m 32s	remaining: 1m 19s
728:	learn: 0.6052558	test: 0.6055907	best: 0.6055907 (728)	total: 3m 33s	remaining: 1m 19s
730:	learn: 0.6052505	test: 0.6055847	best: 0.6055847 (730)	total: 3m 33s	remaining: 1m 18s
732:	learn: 0.6052408	test: 0.6055753	best: 0.6055753 (732)	total: 3m 34s	remaining: 1m 18s
734:	learn: 0.6052277	test: 0.6055607	best: 0.6055607 (734)	total: 3m 35s	remaining: 1m 17s
736:	learn: 0.6052089	test: 0.6055435	best: 0.6055435 (736)	total: 3m 35s	remaining: 1m 16s
738:	learn: 0.6052005	test: 0.6055350	best: 0.6055350 (738)	total: 3m 36s	remaining: 1m 16s
740:	learn: 0.6051976	test: 0.6055308	best: 0.6055308 (740)	total: 3m 36s	remaining: 1m 15s
742:	learn: 0.6051906	test: 0.6055255	best: 0.6055255 (742)	total: 3m 37s	remaining: 1m 15s
744:	learn: 0.6051794	test: 0.6055164	best: 0.6055164 (744)	total: 3m 38s	remain

906:	learn: 0.6043744	test: 0.6047345	best: 0.6047345 (906)	total: 4m 24s	remaining: 27.1s
908:	learn: 0.6043689	test: 0.6047275	best: 0.6047275 (908)	total: 4m 24s	remaining: 26.5s
910:	learn: 0.6043622	test: 0.6047207	best: 0.6047207 (910)	total: 4m 25s	remaining: 25.9s
912:	learn: 0.6043557	test: 0.6047134	best: 0.6047134 (912)	total: 4m 25s	remaining: 25.3s
914:	learn: 0.6043407	test: 0.6046987	best: 0.6046987 (914)	total: 4m 26s	remaining: 24.7s
916:	learn: 0.6043328	test: 0.6046917	best: 0.6046917 (916)	total: 4m 26s	remaining: 24.1s
918:	learn: 0.6043273	test: 0.6046864	best: 0.6046864 (918)	total: 4m 27s	remaining: 23.6s
920:	learn: 0.6043188	test: 0.6046779	best: 0.6046779 (920)	total: 4m 27s	remaining: 23s
922:	learn: 0.6043061	test: 0.6046655	best: 0.6046655 (922)	total: 4m 28s	remaining: 22.4s
924:	learn: 0.6042985	test: 0.6046584	best: 0.6046584 (924)	total: 4m 29s	remaining: 21.8s
926:	learn: 0.6042907	test: 0.6046530	best: 0.6046530 (926)	total: 4m 29s	remaining: 21.2s
9

[I 2021-05-20 13:47:03,257] Trial 29 finished with values: [9.951909593383402, 0.5927728906575979] and parameters: {'objective': 'Logloss', 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.181491316794338}. 


Wall time: 2h 28min 9s


<timed exec>:3: ExperimentalWarning:

plot_pareto_front is experimental (supported from v2.4.0). The interface can change in the future.



In [30]:
study.best_trials

[FrozenTrial(number=25, values=[11.608393805242766, 0.6140421041860606], datetime_start=datetime.datetime(2021, 5, 20, 13, 6, 0, 426405), datetime_complete=datetime.datetime(2021, 5, 20, 13, 8, 41, 946641), params={'objective': 'Logloss', 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.2221925929896766}, distributions={'objective': CategoricalDistribution(choices=('Logloss',)), 'depth': IntUniformDistribution(high=12, low=1, step=1), 'boosting_type': CategoricalDistribution(choices=('Ordered', 'Plain')), 'bootstrap_type': CategoricalDistribution(choices=('Bayesian', 'Bernoulli', 'MVS')), 'subsample': UniformDistribution(high=1.0, low=0.1)}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, trial_id=25, state=TrialState.COMPLETE, value=None)]

In [28]:
best_gbm = tune_lightGBM_sequential(X_train, X_val, y_train, y_val)

[I 2021-05-16 16:46:58,141] A new study created in memory with name: no-name-9f0c5ec9-42cc-4ef8-bb9f-ddc76d163eb9





  0%|          | 0/7 [00:00<?, ?it/s]




feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Starting training lightGBM sequential
[LightGBM] [Info] Number of positive: 2790775, number of negative: 4266576
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.871088
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.310593
[LightGBM] [Debug] init for col-wise cost 0.440455 seconds, init for row-wise cost 1.501745 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.709903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1989
[LightGBM] [Info] Number of data points in the train set: 7057351, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.395442 -> initscore=-0.424492
[LightGBM] [Info] Start training from score -0.424492
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[1]	valid_0's rce: 1.32381

[W 2021-05-16 16:48:33,344] Trial 0 failed because of the following error: KeyError('custom')
Traceback (most recent call last):
  File "C:\Users\Mattia\anaconda3\envs\rec_sys_challenge\lib\site-packages\optuna\_optimize.py", line 217, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Mattia\anaconda3\envs\rec_sys_challenge\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 251, in __call__
    val_score = self._get_booster_best_score(booster)
  File "C:\Users\Mattia\anaconda3\envs\rec_sys_challenge\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 118, in _get_booster_best_score
    val_score = booster.best_score[valid_name][metric]
KeyError: 'custom'


KeyError: 'custom'

In [67]:
import gc
gc.collect()

529

In [68]:
best_xgb = tune_XGBoost(X_train, X_val, y_train, y_val, 2)

[I 2021-05-15 17:39:11,442] A new study created in memory with name: no-name-04bc1a77-4f83-4a76-a199-e04cae044581
[I 2021-05-15 17:40:51,871] Trial 0 finished with values: [-3.280984984544677, 0.5438537032903589] and parameters: {'booster': 'dart', 'lambda': 0.023853561675799762, 'alpha': 5.498002180940041e-08, 'subsample': 0.7107501605504862, 'colsample_bytree': 0.7536655737121543, 'max_depth': 5, 'min_child_weight': 2, 'eta': 6.2185421870689736e-06, 'gamma': 6.684639155146274e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 5.032822722169544e-05, 'skip_drop': 0.004600759483520226}. 
[I 2021-05-15 17:42:12,138] Trial 1 finished with values: [-3.2820318103682133, 0.5503807206128154] and parameters: {'booster': 'gbtree', 'lambda': 5.137678862460926e-08, 'alpha': 2.80260206920225e-06, 'subsample': 0.8332191477348228, 'colsample_bytree': 0.2760825361558681, 'max_depth': 9, 'min_child_weight': 8, 'eta': 6.769009112121167e-07, 'gamma': 2.05